## Prepare

In [6]:
from web3 import Web3, HTTPProvider
from eth_account.messages import encode_defunct
import eth_abi, json
from eth_account import Account

w3 = Web3(HTTPProvider("https://opbnb-mainnet-rpc.bnbchain.org"))

ABI_PATH = '../abi/contracts'
contracts_abi = {
    'masterToken': json.load(open(f'{ABI_PATH}/assets/MasterToken.sol/MasterToken.json')),
    'masterBot': json.load(open(f'{ABI_PATH}/tokenbound/MasterBotNFT.sol/MasterBotNFT.json')),
    'functionBot': json.load(open(f'{ABI_PATH}/assets/FunctionBot.sol/FunctionBot.json')),
    'erc6551registry': json.load(open(f'{ABI_PATH}/tokenbound/ERC6551Registry.sol/ERC6551Registry.json')),
    'masterBotTBA': json.load(open(f'{ABI_PATH}/tokenbound/MasterBotTBA.sol/MasterBotTokenBoundAccount.json')),
    'assets': json.load(open(f'{ABI_PATH}/assets/Assets.sol/Assets.json')),
}
contracts_address = {
    'masterToken': '0x8c05A42432a971F8CD1AFbdeB70f8e732337cE69',
    'masterBot': '0x95c3385c6560f414E3d5927398fA744B27d1F991',
    'functionBot': '0xB5Ad35735D8d78c6812C8299CbB8fe4DeD716e82',
    'erc6551registry': '0xfa09865A069755f144d79440f9DB1bc2f469dA5d',
    'masterBotTBA': '0x68b04C6e349115d282F7b15dc2a319593D7a3483',
    'assets': '0x000BF4bC5328b7364AA0F127f367ae2119A7174A',
}
contracts = {
    key: w3.eth.contract(address=contracts_address[key], abi=contracts_abi[key]) for key in contracts_abi
}

In [2]:
# Chain ID of opBNB is 204. See https://chainlist.org/chain/204.
CHAIN_ID = 204

# The salt value for generate TBA is 0x6551...6551. See https://eips.ethereum.org/EIPS/eip-6551.
SALT = 0x6551655165516551655165516551655165516551655165516551655165516551

# Load your private key.
user_pk = open('privateKey.txt').read()
user_addr = Account.from_key(user_pk).address
print("User address: ", user_addr)

User address:  0x1b47a6F76a21fDee8BeCA56040A698f87aBc1511


In [3]:
# Ask admin to mint you a Masterbot NFT first.
# ...

# Now suppose you're the owner of Masterbot NFT #0
token_id = 0
print(f"Owner of MasterBot #{token_id}: ", contracts['masterBot'].functions.ownerOf(token_id).call())

Owner of MasterBot #0:  0x1b47a6F76a21fDee8BeCA56040A698f87aBc1511


## Register ERC6551 accounts

In [5]:
TBA_address = contracts['erc6551registry'].functions.account(
    contracts_address['masterBotTBA'],      # TBA implementation address
    CHAIN_ID,
    contracts_address['masterBot'],         # MasterBot NFT address
    token_id,
    SALT,
).call()

print(f"TBA address for MasterBot #{token_id}: ", TBA_address)

TBA address for MasterBot #0:  0x40500cD67895f740Fb6B574dF5392f066d3468dF


In [20]:
transaction = {
    'value': 0,
    'gas': 100000,
    'gasPrice': w3.to_wei('14791', 'wei'),
    'nonce': w3.eth.get_transaction_count(user_addr),
}

contract_function = contracts['erc6551registry'].functions.createAccount(
    contracts_address['masterBotTBA'], CHAIN_ID, contracts_address['masterBot'], token_id, SALT, b''
)
signed_transaction = w3.eth.account.sign_transaction(contract_function.build_transaction(transaction), user_pk)
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction complete. Transaction hash:", tx_hash.hex())

Transaction complete. Transaction hash: 0xfb91aa48a8055298c939748c710dc40793c7c1fa7d01c8ca58e1ad58b5eb94b7


## Mint some assets to the TBA

In [28]:
# Ask admin to mint you some MasterToken ($MTT)
# ...

print(f"$MTT balance of TBA address {TBA_address}: ",
      contracts['masterToken'].functions.balanceOf(TBA_address).call() / 1e18)

# Ask admin to mint you some FunctionBot ($FTB)
# ...

print(f"FTB balance of TBA address {TBA_address}: ",
      contracts['functionBot'].functions.balanceOf(TBA_address).call())
print("Token indices for TBA: ",
      contracts['functionBot'].functions.tokenOfOwnerByIndex(TBA_address, 0).call(),
      contracts['functionBot'].functions.tokenOfOwnerByIndex(TBA_address, 1).call())

$MTT balance of TBA address 0x40500cD67895f740Fb6B574dF5392f066d3468dF:  5000.0
FTB balance of TBA address 0x40500cD67895f740Fb6B574dF5392f066d3468dF:  2
Token indices for TBA:  0 1


## Approve for Asset-contract

In [49]:
calldata_approve = contracts['masterToken'] \
    .functions.approve(contracts_address['assets'], int(1000e18)) \
    .build_transaction(transaction)['data']

contract_for_my_TBA = w3.eth.contract(address=TBA_address, abi=contracts_abi['masterBotTBA'])

transaction = {
    'value': 0,
    'gas': 100000,
    'gasPrice': w3.to_wei('14791', 'wei'),
    'nonce': w3.eth.get_transaction_count(user_addr),
}

signed_transaction = w3.eth.account.sign_transaction(contract_for_my_TBA.functions.execute(
    contracts_address['masterToken'],   # the target address
    0,                                  # $BNB value (always 0)
    calldata_approve,                   # calldata of approving for Asset contract
    0,                                  # operation code (always 0)
).build_transaction(transaction), user_pk)
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction complete. Transaction hash:", tx_hash.hex())

Transaction complete. Transaction hash: 0x78671f8771d2b84e2df5e486f7d12697f7d3d4ee7a1ccd1c16b392c33b84d96d


## Use $MTT to interact with Asset-contract

In [55]:
calldata_buy_share = contracts['assets'] \
    .functions.purchase() \
    .build_transaction(transaction)['data']

transaction = {
    'value': 0,
    'gas': 200000,
    'gasPrice': w3.to_wei('14791', 'wei'),
    'nonce': w3.eth.get_transaction_count(user_addr),
}

signed_transaction = w3.eth.account.sign_transaction(contract_for_my_TBA.functions.execute(
    contracts_address['assets'],        # the target address
    0,
    calldata_buy_share,                 # calldata of purchasing a `Share Token`
    0,
).build_transaction(transaction), user_pk)
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction complete. Transaction hash:", tx_hash.hex())

Transaction complete. Transaction hash: 0x76223e895e4e30ef5bf500dc262150cd31b8e47e5f5aabde6fe090712b524155


In [58]:
print(f"$MTT balance of TBA address {TBA_address}: ",
      contracts['masterToken'].functions.balanceOf(TBA_address).call() / 1e18)
print(f"Share balance of TBA address {TBA_address}: ",
      contracts['assets'].functions.balanceShareOf(TBA_address).call())

$MTT balance of TBA address 0x40500cD67895f740Fb6B574dF5392f066d3468dF:  4999.0
Share balance of TBA address 0x40500cD67895f740Fb6B574dF5392f066d3468dF:  1


In [50]:
[(f['name'], f['type']) for f in contracts_abi['assets'] if f['type']=='function']

[('CHARGE_PRICE', 'function'),
 ('ENERGY_REDUCE_FREQ', 'function'),
 ('INSTALL_PRICE', 'function'),
 ('TAX_BASE_POINT', 'function'),
 ('_claimAll', 'function'),
 ('adminBalance', 'function'),
 ('balanceMasterTokenOf', 'function'),
 ('balanceShareOf', 'function'),
 ('charge', 'function'),
 ('charge_price_integer', 'function'),
 ('decimals', 'function'),
 ('energyOf', 'function'),
 ('functionBot', 'function'),
 ('installBot', 'function'),
 ('install_price_integer', 'function'),
 ('isInstalledOn', 'function'),
 ('masterToken', 'function'),
 ('owner', 'function'),
 ('purchase', 'function'),
 ('renounceOwnership', 'function'),
 ('sell', 'function'),
 ('slotsOf', 'function'),
 ('totalSupplyShare', 'function'),
 ('transferOwnership', 'function'),
 ('uninstallBot', 'function')]